# Final Project - Reinforcements Learning 
Hello dear students,<br> this is the template notebook. Please click on the "File" tab and then on "Save a copy into drive".

---
<br>

### Name and ID:
Student 1: Avraham Raviv, 204355390
<br>
Student 2: Yevgeni Berkovitch, 317079234
<br><br>
<img src="https://play-lh.googleusercontent.com/e_oKlKPISbgdzut1H9opevS7-LTB8-8lsmpCdMkhlnqFenZhpjxbLmx7l158-xQQCIY">

### https://github.com/mpSchrader/gym-sokoban

# Installs

In [1]:
%%capture
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install gym
!pip install pygame
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym_sokoban

!imageio_download_bin ffmpeg

# Imports

In [2]:
import random
import time

import numpy as np
import matplotlib.pyplot as plt

import base64
import imageio
from pyvirtualdisplay import Display
from IPython.display import HTML

import gym
from gym import error, spaces, utils
from soko_pap import *

from collections import deque
from queue import PriorityQueue

from keras import Model
from keras.models import Sequential
from keras.layers import Dense

import tensorflow as tf 

In [3]:
%matplotlib inline

In [4]:
imageio.plugins.ffmpeg.download()

In [5]:
from gym import logger as gymlogger
gymlogger.set_level(40) # error only

# Display utils
The cell below contains the video display configuration. No need to make changes here.

In [6]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return HTML(tag)

# Utils

In [7]:
def get_distances(room_state):
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):
            if room_state[i][j] == 2:
                target = (i, j)

    distances = np.zeros(shape=room_state.shape)
    visited_cells = set()
    cell_queue = deque()

    visited_cells.add(target)
    cell_queue.appendleft(target)

    while len(cell_queue) != 0:
        cell = cell_queue.pop()
        distance = distances[cell[0]][cell[1]]
        for x,y in ((1,0), (-1,-0), (0,1), (0,-1)):
            next_cell_x, next_cell_y = cell[0]+x, cell[1]+y
            if room_state[next_cell_x][next_cell_y] != 0 and not (next_cell_x, next_cell_y) in visited_cells:
                distances[next_cell_x][next_cell_y] = distance + 1
                visited_cells.add((next_cell_x, next_cell_y))
                cell_queue.appendleft((next_cell_x, next_cell_y))
                
    return distances   

def calc_distances(room_state, distances):
    box = None
    mover = None
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):            
            if room_state[i][j] == 4:
                box = (i,j)
            
            if room_state[i][j] == 5:
                mover = (i,j)
    
    return mover, box, distances[box[0]][box[1]]   

def box2target_change_reward(room_state, next_room_state, distances):
    if np.array_equal(room_state, next_room_state):
        return -1.0
    
    mover, box, t2b = calc_distances(room_state, distances)
    n_mover, n_box, n_t2b = calc_distances(next_room_state, distances)
    
    change_reward = 0.0
    if n_t2b < t2b:
        change_reward += 5.0
    elif n_t2b > t2b:
        change_reward -= 5.0
        
    m2b = np.sqrt((mover[0]-box[0])**2 + (mover[1]-box[1])**2)
    n_m2b = np.sqrt((n_mover[0]-n_box[0])**2 + (n_mover[1]-n_box[1])**2)
    
    if n_m2b < m2b and m2b >= 2:
        change_reward += 1.0
    elif n_m2b > m2b and n_m2b >= 2:
        change_reward -= 1.0
        
    return change_reward   

# Solution

In [30]:
class DuelingDQN(Model):
    def __init__(self, n_actions):
        super(DuelingDQN, self).__init__()
        self.dense1 = Dense(512, activation='relu')
        self.dense2 = Dense(64, activation='relu')
        self.V = Dense(1, activation=None)
        self.A = Dense(n_actions, activation=None)
        
    def call(self, state):
        x = self.dense1(state)
        x = self.dense2(x)
        V = self.V(x)
        A = self.A(x)
        Q = V + (A - tf.math.reduce_mean(A, axis=1, keepdims=True))
        return Q
    
    def advantage(self, state):
        x = self.dense1(state)
        x = self.dense2(x)
        A = self.A(x)
        return A

In [31]:
class SOK_Agent:
    def __init__(self):
        # Dimensions
        self.state_size = (25,) 
        self.action_size = 8        
        self.batch_size = 8
        
        # Replay buffers
        self.replay_buffer = deque(maxlen=5000)
        self.prioritized_replay_buffer = deque(maxlen=500)
        self.prioritized_replay_batch = 50        
        
        # Hyperparameters
        self.gamma = 0.9
        self.epsilon = 1.0   
        self.epsilon_min = 0.3
        self.epsilon_decay = 0.995
        self.replay_rate = 10
        self.update_beta = 0.999

        self.verbosity = 100 
        
        # Models
        self.model = DuelingDQN(self.action_size)
        self.target_model = DuelingDQN(self.action_size)
        self.target_model.set_weights(self.model.get_weights())
        
        self.model.compile(loss='mse', optimizer="adam")
        self.target_model.compile(loss='mse', optimizer="adam")

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append([state, action, reward, next_state, done])    
        
    def copy_to_prioritized_buffer(self, n):
        for i in range(n):
            self.prioritized_replay_buffer.append(self.replay_buffer[-1-i])  

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0]) 

    def replay(self):        
        minibatch = random.sample(self.replay_buffer, self.batch_size//2) 
        minibatch.extend(random.sample(self.prioritized_replay_buffer, self.batch_size//2))
        
        states = np.zeros((self.batch_size, self.state_size[0]))
        actions = np.zeros(self.batch_size, dtype=int)
        rewards = np.zeros(self.batch_size)
        next_states = np.zeros((self.batch_size, self.state_size[0]))
        statuses = np.zeros(self.batch_size)
                
        for i, (state, action, reward, next_state, done) in enumerate(minibatch): 
            states[i] = state.copy()
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state.copy()
            statuses[i] = 1 if done else 0    
        
        q_pred = self.model.predict(states) 
        q_next = self.target_model.predict(next_states)
        
        q_target = q_pred.copy()
        max_actions = tf.math.argmax(self.model.predict(next_states), axis=1)
        
        for (idx, terminal) in enumerate(statuses):
            q_target[idx, actions[idx]] = rewards[idx] + self.gamma * q_next[idx, max_actions[idx]] * (1 - int(statuses[idx]))
         
        self.model.fit(states, q_target, epochs=1, verbose=0) 
        
    def update_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
        
    def update_target_model(self):
        model_w = self.model.get_weights()
        target_model_w = self.target_model.get_weights()
        updated_target_model_w = []
        for i in range(len(model_w)):
            updated_target_model_w.append(self.update_beta*target_model_w[i] + (1-self.update_beta)*model_w[i])
        self.target_model.set_weights(updated_target_model_w)    
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [32]:
def process_frame(frame):
    f = frame[16:96, 16:96, 0]   
    f = f.reshape(5, 16, 5, 16).max(axis=(1, 3))
    f = f.flatten()
    f = f / 255
    return np.expand_dims(f, axis=0)

## Training

In [33]:
max_episodes = 10000
max_steps = 100

def init_sok(r):
    random.seed(r%10)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(max_steps)
    return sok

In [34]:
agent = SOK_Agent()
successes_before_train = 20
successful_episodes = 0
continuous_successes_goal = 100
continuous_successes = 0

steps_per_episode = []

for e in range(max_episodes):
    if continuous_successes >= continuous_successes_goal:
        print("Agent training finished!")
        break
    
    print("Episode: %d" % (e))
    
    sok = init_sok(e)
    state = process_frame(sok.get_image('rgb_array'))
    random.seed(e)
    
    room_state = sok.room_state.copy() 
    distances = get_distances(room_state)
    
    for step in range(sok.max_steps):
        action = agent.act(state)
        if action < 4:
            next_state, reward, done, _ = sok.step(action+1) 
        else:
            next_state, reward, done, _ = sok.step(action+5)         
        
        next_state = process_frame(next_state)        
        next_room_state = sok.room_state
        
        if not done:
            reward += box2target_change_reward(room_state, next_room_state, distances)
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state.copy() 
        room_state = next_room_state.copy()
                
        if successful_episodes >= successes_before_train:
            if (step+1) % agent.replay_rate == 0:
                agent.replay() 
                agent.update_target_model()
                agent.update_epsilon()
        
        if done:            
            if 3 in sok.room_state:
                successful_episodes += 1
                continuous_successes += 1
                print("SOLVED! Episode %d Steps: %d Epsilon %.4f" % (e, step+1, agent.epsilon))                 
            else:
                agent.copy_to_prioritized_buffer(min(agent.prioritized_replay_batch, step+1))
                continuous_successes = 0
                
            steps_per_episode.append(step+1)
            #agent.save("exp1_episode%d.h5" % (e))
            
            break

Episode: 0
Episode: 1
Episode: 2
Episode: 3
SOLVED! Episode 3 Steps: 54 Epsilon 1.0000
Episode: 4
SOLVED! Episode 4 Steps: 1 Epsilon 1.0000
Episode: 5
SOLVED! Episode 5 Steps: 28 Epsilon 1.0000
Episode: 6
Episode: 7
SOLVED! Episode 7 Steps: 25 Epsilon 1.0000
Episode: 8
Episode: 9
SOLVED! Episode 9 Steps: 74 Epsilon 1.0000
Episode: 10
Episode: 11
SOLVED! Episode 11 Steps: 53 Epsilon 1.0000
Episode: 12
Episode: 13
Episode: 14
SOLVED! Episode 14 Steps: 2 Epsilon 1.0000
Episode: 15
Episode: 16
Episode: 17
Episode: 18
SOLVED! Episode 18 Steps: 6 Epsilon 1.0000
Episode: 19
Episode: 20
Episode: 21
Episode: 22
Episode: 23
SOLVED! Episode 23 Steps: 2 Epsilon 1.0000
Episode: 24
SOLVED! Episode 24 Steps: 3 Epsilon 1.0000
Episode: 25
Episode: 26
SOLVED! Episode 26 Steps: 44 Epsilon 1.0000
Episode: 27
SOLVED! Episode 27 Steps: 41 Epsilon 1.0000
Episode: 28
SOLVED! Episode 28 Steps: 9 Epsilon 1.0000
Episode: 29
Episode: 30
Episode: 31
Episode: 32
Episode: 33
Episode: 34
Episode: 35
Episode: 36
Episo

Episode: 250
Episode: 251
SOLVED! Episode 251 Steps: 3 Epsilon 0.2988
Episode: 252
Episode: 253
SOLVED! Episode 253 Steps: 1 Epsilon 0.2988
Episode: 254
SOLVED! Episode 254 Steps: 1 Epsilon 0.2988
Episode: 255
SOLVED! Episode 255 Steps: 100 Epsilon 0.2988
Episode: 256
Episode: 257
SOLVED! Episode 257 Steps: 3 Epsilon 0.2988
Episode: 258
SOLVED! Episode 258 Steps: 2 Epsilon 0.2988
Episode: 259
Episode: 260
Episode: 261
SOLVED! Episode 261 Steps: 3 Epsilon 0.2988
Episode: 262
Episode: 263
SOLVED! Episode 263 Steps: 1 Epsilon 0.2988
Episode: 264
SOLVED! Episode 264 Steps: 1 Epsilon 0.2988
Episode: 265
Episode: 266
SOLVED! Episode 266 Steps: 56 Epsilon 0.2988
Episode: 267
SOLVED! Episode 267 Steps: 3 Epsilon 0.2988
Episode: 268
SOLVED! Episode 268 Steps: 2 Epsilon 0.2988
Episode: 269
Episode: 270
Episode: 271
SOLVED! Episode 271 Steps: 3 Epsilon 0.2988
Episode: 272
Episode: 273
SOLVED! Episode 273 Steps: 1 Epsilon 0.2988
Episode: 274
Episode: 275
Episode: 276
SOLVED! Episode 276 Steps: 27 

SOLVED! Episode 443 Steps: 1 Epsilon 0.2988
Episode: 444
SOLVED! Episode 444 Steps: 1 Epsilon 0.2988
Episode: 445
SOLVED! Episode 445 Steps: 25 Epsilon 0.2988
Episode: 446
Episode: 447
SOLVED! Episode 447 Steps: 3 Epsilon 0.2988
Episode: 448
SOLVED! Episode 448 Steps: 2 Epsilon 0.2988
Episode: 449
Episode: 450
SOLVED! Episode 450 Steps: 3 Epsilon 0.2988
Episode: 451
SOLVED! Episode 451 Steps: 3 Epsilon 0.2988
Episode: 452
Episode: 453
SOLVED! Episode 453 Steps: 1 Epsilon 0.2988
Episode: 454
SOLVED! Episode 454 Steps: 1 Epsilon 0.2988
Episode: 455
SOLVED! Episode 455 Steps: 28 Epsilon 0.2988
Episode: 456
SOLVED! Episode 456 Steps: 35 Epsilon 0.2988
Episode: 457
SOLVED! Episode 457 Steps: 3 Epsilon 0.2988
Episode: 458
SOLVED! Episode 458 Steps: 2 Epsilon 0.2988
Episode: 459
Episode: 460
SOLVED! Episode 460 Steps: 3 Epsilon 0.2988
Episode: 461
SOLVED! Episode 461 Steps: 3 Epsilon 0.2988
Episode: 462
Episode: 463
SOLVED! Episode 463 Steps: 1 Epsilon 0.2988
Episode: 464
SOLVED! Episode 464 

Episode: 630
SOLVED! Episode 630 Steps: 3 Epsilon 0.2988
Episode: 631
SOLVED! Episode 631 Steps: 3 Epsilon 0.2988
Episode: 632
Episode: 633
SOLVED! Episode 633 Steps: 1 Epsilon 0.2988
Episode: 634
SOLVED! Episode 634 Steps: 1 Epsilon 0.2988
Episode: 635
Episode: 636
SOLVED! Episode 636 Steps: 16 Epsilon 0.2988
Episode: 637
SOLVED! Episode 637 Steps: 3 Epsilon 0.2988
Episode: 638
SOLVED! Episode 638 Steps: 2 Epsilon 0.2988
Episode: 639
Episode: 640
SOLVED! Episode 640 Steps: 3 Epsilon 0.2988
Episode: 641
SOLVED! Episode 641 Steps: 3 Epsilon 0.2988
Episode: 642
Episode: 643
SOLVED! Episode 643 Steps: 1 Epsilon 0.2988
Episode: 644
SOLVED! Episode 644 Steps: 1 Epsilon 0.2988
Episode: 645
Episode: 646
SOLVED! Episode 646 Steps: 16 Epsilon 0.2988
Episode: 647
SOLVED! Episode 647 Steps: 3 Epsilon 0.2988
Episode: 648
SOLVED! Episode 648 Steps: 2 Epsilon 0.2988
Episode: 649
Episode: 650
SOLVED! Episode 650 Steps: 3 Epsilon 0.2988
Episode: 651
SOLVED! Episode 651 Steps: 3 Epsilon 0.2988
Episode:

SOLVED! Episode 811 Steps: 3 Epsilon 0.2988
Episode: 812
Episode: 813
SOLVED! Episode 813 Steps: 1 Epsilon 0.2988
Episode: 814
SOLVED! Episode 814 Steps: 1 Epsilon 0.2988
Episode: 815
SOLVED! Episode 815 Steps: 67 Epsilon 0.2988
Episode: 816
SOLVED! Episode 816 Steps: 18 Epsilon 0.2988
Episode: 817
SOLVED! Episode 817 Steps: 3 Epsilon 0.2988
Episode: 818
SOLVED! Episode 818 Steps: 2 Epsilon 0.2988
Episode: 819
Episode: 820
SOLVED! Episode 820 Steps: 3 Epsilon 0.2988
Episode: 821
SOLVED! Episode 821 Steps: 3 Epsilon 0.2988
Episode: 822
Episode: 823
SOLVED! Episode 823 Steps: 1 Epsilon 0.2988
Episode: 824
SOLVED! Episode 824 Steps: 1 Epsilon 0.2988
Episode: 825
Episode: 826
SOLVED! Episode 826 Steps: 26 Epsilon 0.2988
Episode: 827
SOLVED! Episode 827 Steps: 3 Epsilon 0.2988
Episode: 828
SOLVED! Episode 828 Steps: 2 Epsilon 0.2988
Episode: 829
Episode: 830
SOLVED! Episode 830 Steps: 3 Epsilon 0.2988
Episode: 831
SOLVED! Episode 831 Steps: 3 Epsilon 0.2988
Episode: 832
Episode: 833
SOLVED!

...

SOLVED! Episode 9368 Steps: 2 Epsilon 0.2988
Episode: 9369
Episode: 9370
SOLVED! Episode 9370 Steps: 3 Epsilon 0.2988
Episode: 9371
SOLVED! Episode 9371 Steps: 3 Epsilon 0.2988
Episode: 9372
SOLVED! Episode 9372 Steps: 9 Epsilon 0.2988
Episode: 9373
SOLVED! Episode 9373 Steps: 1 Epsilon 0.2988
Episode: 9374
SOLVED! Episode 9374 Steps: 1 Epsilon 0.2988
Episode: 9375
SOLVED! Episode 9375 Steps: 7 Epsilon 0.2988
Episode: 9376
SOLVED! Episode 9376 Steps: 6 Epsilon 0.2988
Episode: 9377
SOLVED! Episode 9377 Steps: 3 Epsilon 0.2988
Episode: 9378
SOLVED! Episode 9378 Steps: 2 Epsilon 0.2988
Episode: 9379
Episode: 9380
SOLVED! Episode 9380 Steps: 3 Epsilon 0.2988
Episode: 9381
SOLVED! Episode 9381 Steps: 3 Epsilon 0.2988
Episode: 9382
SOLVED! Episode 9382 Steps: 9 Epsilon 0.2988
Episode: 9383
SOLVED! Episode 9383 Steps: 1 Epsilon 0.2988
Episode: 9384
SOLVED! Episode 9384 Steps: 1 Epsilon 0.2988
Episode: 9385
SOLVED! Episode 9385 Steps: 7 Epsilon 0.2988
Episode: 9386
SOLVED! Episode 9386 Steps: 

SOLVED! Episode 9521 Steps: 3 Epsilon 0.2988
Episode: 9522
SOLVED! Episode 9522 Steps: 9 Epsilon 0.2988
Episode: 9523
SOLVED! Episode 9523 Steps: 1 Epsilon 0.2988
Episode: 9524
SOLVED! Episode 9524 Steps: 1 Epsilon 0.2988
Episode: 9525
SOLVED! Episode 9525 Steps: 28 Epsilon 0.2988
Episode: 9526
SOLVED! Episode 9526 Steps: 32 Epsilon 0.2988
Episode: 9527
SOLVED! Episode 9527 Steps: 3 Epsilon 0.2988
Episode: 9528
SOLVED! Episode 9528 Steps: 2 Epsilon 0.2988
Episode: 9529
Episode: 9530
SOLVED! Episode 9530 Steps: 3 Epsilon 0.2988
Episode: 9531
SOLVED! Episode 9531 Steps: 3 Epsilon 0.2988
Episode: 9532
SOLVED! Episode 9532 Steps: 9 Epsilon 0.2988
Episode: 9533
SOLVED! Episode 9533 Steps: 1 Epsilon 0.2988
Episode: 9534
SOLVED! Episode 9534 Steps: 1 Epsilon 0.2988
Episode: 9535
SOLVED! Episode 9535 Steps: 7 Epsilon 0.2988
Episode: 9536
SOLVED! Episode 9536 Steps: 6 Epsilon 0.2988
Episode: 9537
SOLVED! Episode 9537 Steps: 3 Epsilon 0.2988
Episode: 9538
SOLVED! Episode 9538 Steps: 2 Epsilon 0.

SOLVED! Episode 9667 Steps: 3 Epsilon 0.2988
Episode: 9668
SOLVED! Episode 9668 Steps: 2 Epsilon 0.2988
Episode: 9669
Episode: 9670
SOLVED! Episode 9670 Steps: 3 Epsilon 0.2988
Episode: 9671
SOLVED! Episode 9671 Steps: 3 Epsilon 0.2988
Episode: 9672
SOLVED! Episode 9672 Steps: 9 Epsilon 0.2988
Episode: 9673
SOLVED! Episode 9673 Steps: 1 Epsilon 0.2988
Episode: 9674
SOLVED! Episode 9674 Steps: 1 Epsilon 0.2988
Episode: 9675
SOLVED! Episode 9675 Steps: 7 Epsilon 0.2988
Episode: 9676
SOLVED! Episode 9676 Steps: 6 Epsilon 0.2988
Episode: 9677
SOLVED! Episode 9677 Steps: 3 Epsilon 0.2988
Episode: 9678
SOLVED! Episode 9678 Steps: 2 Epsilon 0.2988
Episode: 9679
Episode: 9680
SOLVED! Episode 9680 Steps: 3 Epsilon 0.2988
Episode: 9681
SOLVED! Episode 9681 Steps: 3 Epsilon 0.2988
Episode: 9682
SOLVED! Episode 9682 Steps: 9 Epsilon 0.2988
Episode: 9683
SOLVED! Episode 9683 Steps: 1 Epsilon 0.2988
Episode: 9684
SOLVED! Episode 9684 Steps: 1 Epsilon 0.2988
Episode: 9685
SOLVED! Episode 9685 Steps: 

SOLVED! Episode 9816 Steps: 6 Epsilon 0.2988
Episode: 9817
SOLVED! Episode 9817 Steps: 3 Epsilon 0.2988
Episode: 9818
SOLVED! Episode 9818 Steps: 2 Epsilon 0.2988
Episode: 9819
Episode: 9820
SOLVED! Episode 9820 Steps: 3 Epsilon 0.2988
Episode: 9821
SOLVED! Episode 9821 Steps: 3 Epsilon 0.2988
Episode: 9822
SOLVED! Episode 9822 Steps: 9 Epsilon 0.2988
Episode: 9823
SOLVED! Episode 9823 Steps: 1 Epsilon 0.2988
Episode: 9824
SOLVED! Episode 9824 Steps: 1 Epsilon 0.2988
Episode: 9825
SOLVED! Episode 9825 Steps: 7 Epsilon 0.2988
Episode: 9826
SOLVED! Episode 9826 Steps: 6 Epsilon 0.2988
Episode: 9827
SOLVED! Episode 9827 Steps: 3 Epsilon 0.2988
Episode: 9828
SOLVED! Episode 9828 Steps: 2 Epsilon 0.2988
Episode: 9829
Episode: 9830
SOLVED! Episode 9830 Steps: 3 Epsilon 0.2988
Episode: 9831
SOLVED! Episode 9831 Steps: 3 Epsilon 0.2988
Episode: 9832
SOLVED! Episode 9832 Steps: 9 Epsilon 0.2988
Episode: 9833
SOLVED! Episode 9833 Steps: 1 Epsilon 0.2988
Episode: 9834
SOLVED! Episode 9834 Steps: 

SOLVED! Episode 9967 Steps: 3 Epsilon 0.2988
Episode: 9968
SOLVED! Episode 9968 Steps: 2 Epsilon 0.2988
Episode: 9969
Episode: 9970
SOLVED! Episode 9970 Steps: 3 Epsilon 0.2988
Episode: 9971
SOLVED! Episode 9971 Steps: 3 Epsilon 0.2988
Episode: 9972
Episode: 9973
SOLVED! Episode 9973 Steps: 1 Epsilon 0.2988
Episode: 9974
SOLVED! Episode 9974 Steps: 1 Epsilon 0.2988
Episode: 9975
SOLVED! Episode 9975 Steps: 7 Epsilon 0.2988
Episode: 9976
SOLVED! Episode 9976 Steps: 6 Epsilon 0.2988
Episode: 9977
SOLVED! Episode 9977 Steps: 3 Epsilon 0.2988
Episode: 9978
SOLVED! Episode 9978 Steps: 2 Epsilon 0.2988
Episode: 9979
Episode: 9980
SOLVED! Episode 9980 Steps: 3 Epsilon 0.2988
Episode: 9981
SOLVED! Episode 9981 Steps: 3 Epsilon 0.2988
Episode: 9982
SOLVED! Episode 9982 Steps: 9 Epsilon 0.2988
Episode: 9983
SOLVED! Episode 9983 Steps: 1 Epsilon 0.2988
Episode: 9984
SOLVED! Episode 9984 Steps: 1 Epsilon 0.2988
Episode: 9985
SOLVED! Episode 9985 Steps: 7 Epsilon 0.2988
Episode: 9986
SOLVED! Episod

## Test Generalization

#### Learned Policy

In [37]:
agent.epsilon = 0.0
num_solved = 0

for t in range(100):    
    random.seed(t+10)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(100)
    
    steps = 0
        
    state = sok.get_image('rgb_array')
    done = False
    while not done:
        steps += 1
        action = agent.act(process_frame(state))

        if action < 4:
            action += 1
        else:
            action += 5

        state, reward, done, info = sok.step(action)
        
    solved = False
    if 3 in sok.room_state:
        solved = True
        num_solved += 1
    
    print("Puzzle # %d: %d (%s) [%d/%d]" % ((t+1), steps, "YES" if solved else "NOT", num_solved, (t+1)))

Puzzle # 1: 100 (NOT) [0/1]
Puzzle # 2: 100 (NOT) [0/2]
Puzzle # 3: 100 (NOT) [0/3]
Puzzle # 4: 100 (NOT) [0/4]
Puzzle # 5: 100 (NOT) [0/5]
Puzzle # 6: 1 (YES) [1/6]
Puzzle # 7: 100 (NOT) [1/7]
Puzzle # 8: 100 (NOT) [1/8]
Puzzle # 9: 100 (NOT) [1/9]
Puzzle # 10: 100 (NOT) [1/10]
Puzzle # 11: 100 (NOT) [1/11]
Puzzle # 12: 100 (NOT) [1/12]
Puzzle # 13: 100 (NOT) [1/13]
Puzzle # 14: 100 (NOT) [1/14]
Puzzle # 15: 100 (NOT) [1/15]
Puzzle # 16: 100 (NOT) [1/16]
Puzzle # 17: 100 (NOT) [1/17]
Puzzle # 18: 100 (NOT) [1/18]
Puzzle # 19: 100 (NOT) [1/19]
Puzzle # 20: 100 (NOT) [1/20]
Puzzle # 21: 100 (NOT) [1/21]
Puzzle # 22: 100 (NOT) [1/22]
Puzzle # 23: 100 (NOT) [1/23]
Puzzle # 24: 100 (NOT) [1/24]
Puzzle # 25: 100 (NOT) [1/25]
Puzzle # 26: 2 (YES) [2/26]
Puzzle # 27: 100 (NOT) [2/27]
Puzzle # 28: 1 (YES) [3/28]
Puzzle # 29: 100 (NOT) [3/29]
Puzzle # 30: 100 (NOT) [3/30]
Puzzle # 31: 3 (YES) [4/31]
Puzzle # 32: 100 (NOT) [4/32]
Puzzle # 33: 100 (NOT) [4/33]
Puzzle # 34: 100 (NOT) [4/34]
Puzzle

#### Random Policy

In [38]:
agent.epsilon = 1.0
num_solved = 0

for t in range(100):    
    random.seed(t+10)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(100)
    
    steps = 0
        
    state = sok.get_image('rgb_array')
    done = False
    while not done:
        steps += 1
        action = agent.act(process_frame(state))

        if action < 4:
            action += 1
        else:
            action += 5

        state, reward, done, info = sok.step(action)
        
    solved = False
    if 3 in sok.room_state:
        solved = True
        num_solved += 1
    
    print("Puzzle # %d: %d (%s) [%d/%d]" % ((t+1), steps, "YES" if solved else "NOT", num_solved, (t+1)))

Puzzle # 1: 100 (NOT) [0/1]
Puzzle # 2: 100 (NOT) [0/2]
Puzzle # 3: 17 (YES) [1/3]
Puzzle # 4: 100 (NOT) [1/4]
Puzzle # 5: 100 (NOT) [1/5]
Puzzle # 6: 100 (NOT) [1/6]
Puzzle # 7: 16 (YES) [2/7]
Puzzle # 8: 1 (YES) [3/8]
Puzzle # 9: 100 (NOT) [3/9]
Puzzle # 10: 100 (NOT) [3/10]
Puzzle # 11: 100 (NOT) [3/11]
Puzzle # 12: 12 (YES) [4/12]
Puzzle # 13: 100 (NOT) [4/13]
Puzzle # 14: 100 (NOT) [4/14]
Puzzle # 15: 100 (NOT) [4/15]
Puzzle # 16: 100 (NOT) [4/16]
Puzzle # 17: 43 (YES) [5/17]
Puzzle # 18: 100 (NOT) [5/18]
Puzzle # 19: 100 (NOT) [5/19]
Puzzle # 20: 17 (YES) [6/20]
Puzzle # 21: 100 (NOT) [6/21]
Puzzle # 22: 100 (NOT) [6/22]
Puzzle # 23: 100 (NOT) [6/23]
Puzzle # 24: 100 (NOT) [6/24]
Puzzle # 25: 100 (NOT) [6/25]
Puzzle # 26: 100 (NOT) [6/26]
Puzzle # 27: 100 (NOT) [6/27]
Puzzle # 28: 100 (NOT) [6/28]
Puzzle # 29: 100 (NOT) [6/29]
Puzzle # 30: 3 (YES) [7/30]
Puzzle # 31: 100 (NOT) [7/31]
Puzzle # 32: 100 (NOT) [7/32]
Puzzle # 33: 100 (NOT) [7/33]
Puzzle # 34: 56 (YES) [8/34]
Puzzle #